# Грузим библиотеки

In [ ]:
from flask import Flask
from flask import request
import requests
from flask import make_response
import os
import json
from pandas import DataFrame
import traceback


from methods import access
from methods.emoji import emoji
from methods import telegram_bot_methods
from methods import google_maps_methods
from methods import psql_methods
from methods import reply



import app



if __name__ == "__main__":

    
    app.application.run()



 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Traceback (most recent call last):
  File "D:\Python\telegram_bots\family_budget\app.py", line 336, in main
    r = psql_methods.add_transaction_plan_month(chat_id = chat_id , summa = summa , dict_user_data = dict_user_data)
  File "D:\Python\telegram_bots\family_budget\methods\psql_methods.py", line 229, in add_transaction_plan_month
    cur.execute("INSERT INTO public.month_transaction_plan (user_id,wallet_id,day,transaction_name,transaction_type,summa) VALUES (%(user_id)s,%(wallet_id)s, %(plan_day)s,'%(transaction_name)s','%(transaction_type)s' , %(summa)s  )" % {'user_id' : user_id , 'wallet_id' : personal_wallet_id , 'transaction_type' : plan_type , 'transaction_name' : plan_name , 'summa' : summa , 'plan_day' : plan_day}  )
psycopg2.DataError: invalid input syntax for integer: "долги"
LINE 1: ...summa) VALUES (13,5, 25,'Платеж по кредиту Олега','долги' , ...
                                                             ^


In [ ]:
X = {'update_id': 201011794, 'message': {'message_id': 159, 'from': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'language_code': 'ru-RU'}, 'chat': {'id': 84723474, 'first_name': 'Александр', 'last_name': 'Толмачёв', 'type': 'private'}, 'date': 1502611993, 'text': '/help', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 5}]}}
X['message']['text'] = '/start'
X['message']['chat']['last_name']

In [ ]:
#тест отправки графика
import psycopg2
from pandas import DataFrame
import pandas as pd
from datetime import datetime
from datetime import timedelta

#завтрашная дата без часов минут
def tomorrow_str():
    tomorrow = datetime.now()+ timedelta(days=1)
    tomorrow_str = str(tomorrow.year)+str(tomorrow.month if tomorrow.month >= 10 else  '0'+str(tomorrow.month))+str(tomorrow.day if tomorrow.day >= 10 else  '0'+str(tomorrow.day)) 
    return tomorrow_str

now = datetime.now()

dbname = 'dbr3jigs1op5oo'
port = '5432'
user = 'muwrkppfuyldmk'
host = 'ec2-54-227-252-202.compute-1.amazonaws.com'
password = '4c4eabfcaf92f7289ccfc1a314d04a3c3806db72b1bf12fd5f0f40c410b14355'

PSQL_heroku_keys = {'dbname' : dbname
                    , 'port' : port
                    , 'user' : user
                    , 'host' : host
                    , 'password' : password
                    }


user_id = 13


conn = psycopg2.connect("dbname='%(dbname)s' port='%(port)s' user='%(user)s' host='%(host)s' password='%(password)s'" % PSQL_heroku_keys)

response = {'status' : 200
            ,'report' : 'tomorrow_expense'
            ,'system_message' : 'No reports'
            ,'user_messages' : []
            }
###################
cur = conn.cursor()
#смотрим, какие данные завтра есть
cur.execute("select * from public.transaction_fact where user_id = %(user_id)s and date_fact >= date_trunc('month', now()) " % {'user_id' : user_id} )



#получаем данные
df_current_fact = DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

print ('ok')

In [ ]:
df_current_fact['day'] = df_current_fact['date_fact'].dt.day
df_current_fact.groupby(['day'])['summa'].sum().reset_index()[:]


import numpy as np
import matplotlib.pyplot as plt

N = 5
men_means = (20, 35, 30, 35, 27)
men_std = (2, 3, 4, 1, 2)

ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind, men_means, width, color='r', yerr=men_std)

ax.set_ylabel('Сумма (р.)')
ax.set_title('Траты в текущем месяце')

plt.show()